## Inverted Index Construction
Name: Suyash Chintawar

Roll No.: 191IT109

Title: IT458 Assignment 1

In [1]:
# Install required external libraries
!pip install requests
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re
import os
import nltk
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

## Web scraping to generate text corpus

In [3]:
# Get links from where the documents will be extracted

root = 'https://www.incredibleindia.org/'
cities = ['delhi','bengaluru','kolkata','varanasi','hyderabad','jaipur','udaipur']
links = []

for city in cities:
  city_page_url = 'https://www.incredibleindia.org/content/incredible-india-v2/en/destinations/' + city + '.html'
  page = requests.get(city_page_url)
  soup = BeautifulSoup(page.content, 'html.parser')

  s = soup.find_all('div', class_='immersive-experience-item')
  for category in s:
    link = category.find('a')['href']
    link = root + link
    links.append(link)

In [4]:
links

['https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/delhi/heritage.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/delhi/spiritual.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/delhi/food-and-cuisine.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/bengaluru/nature.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/bengaluru/food-and-cuisine.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/bengaluru/art-and-crafts.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/kolkata/food-and-cuisine.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/kolkata/nature.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/kolkata/arts-and-crafts.html',
 'https://www.incredibleindia.org//cont

In [5]:
!rm -rf data
!mkdir data

In [6]:
# Extracting the documents from the links above

count = 0
for url in links:
  page = requests.get(url)
  soup = BeautifulSoup(page.content,'html.parser')
  s = soup.find_all('div',class_='listicle aem-GridColumn aem-GridColumn--default--12')
  for item in s:
    title = item.find('h2',class_='title').text
    title = re.sub('[^a-zA-Z0-9 ]','',title)
    title = '-'.join(title.lower().split())
    content = item.find('p').text
    content = ' '.join(content.split())

    ofile = open('data/'+title+'.txt','w')
    ofile.write(content)
    ofile.close()

    count+=1

print('Number of files generated:',count)

Number of files generated: 151


## PART A: Preprocessing on the text corpus

In [7]:
# Part A
# Preprocessing using different techniques

#Get file names and their texts
path = 'data/'
files = []
texts = []
for file_name in os.listdir(path):
  file = open(path+file_name,'r')
  files.append(file_name)
  texts.append(file.read())

files = files[:100]
texts = texts[:100]

print('Files: ',files)
print('Files length:',len(files))
print('Texts: ',texts)
print('Texts length:',len(texts))

Files:  ['ekling-temple.txt', 'mecca-masjid.txt', 'paigah-tombs.txt', 'mirchi-vada.txt', 'manikarnika-ghat.txt', 'nandi-hills.txt', 'pochampally-sarees.txt', 'kal-bhairava-temple.txt', 'sandalwood-artefacts.txt', 'parantha.txt', 'phuchka.txt', 'gujiya.txt', 'bisi-bele-baath.txt', 'dal-baati-churma.txt', 'irani-chai.txt', 'qutub-shahi-tombs.txt', 'parshavnath-jain-temple.txt', 'ker-sangri.txt', 'dosa.txt', 'butter-chicken.txt', 'jama-masjid.txt', 'bagru.txt', 'jag-mandir-palace.txt', 'desserts-and-sweets.txt', 'national-war-memorial.txt', 'sheer-korma.txt', 'india-gate.txt', 'osmania-biscuit.txt', 'jhall-muri.txt', 'mukutmanipur.txt', 'chhole-bhature.txt', 'kolkata-biryani.txt', 'luchi.txt', 'old-fort-purana-quila.txt', 'leheriya-and-bandhej.txt', 'mughlai-paratha.txt', 'humayuns-tomb.txt', 'qutub-minar.txt', 'parliament-house.txt', 'alipore-zoological-garden.txt', 'assi-ghat.txt', 'cubbon-park.txt', 'muthyala-maduvu-or-pearl-valley.txt', 'ghugni.txt', 'mutton-biryani.txt', 'ulsoor-lake

In [8]:
# Counting total index terms in the corpus
def count_all_tokens(texts):
  all_tokens = set()
  for text in texts:
    for token in text:
      all_tokens.add(token)
  return len(all_tokens)

In [9]:
# Tokenization
def tokenize(text):
  return text.split()

In [10]:
# Nromalization
def normalize(text):
  new_tokens = []
  for token in text:
    token = re.sub('[^a-zA-Z0-9]','',token)
    token = token.lower()
    new_tokens.append(token)
  return new_tokens

In [11]:
from nltk.stem import WordNetLemmatizer

# Lemmatization
def lemmatize(text):
  lemmatized_text = []
  lemmatizer = WordNetLemmatizer()
  for token in text:
    lemmatized_token = lemmatizer.lemmatize(token)
    lemmatized_text.append(lemmatized_token)
  return lemmatized_text

In [12]:
from nltk.corpus import stopwords

# Stopword removal
def remove_stopwords(text):
  stopwords_list = stopwords.words('english')
  new_tokens = []
  for token in text:
    if token not in stopwords_list:
      new_tokens.append(token)
  return new_tokens

In [15]:
tokenized_texts = []
for text in texts:
  tokenized_texts.append(tokenize(text))
print('Tokenized texts:\n',tokenized_texts)
print('Number of tokens after tokenization:',count_all_tokens(tokenized_texts),end='\n\n')

normalized_texts = []
for text in tokenized_texts:
  normalized_texts.append(normalize(text))
print('Normalized texts:\n',normalized_texts)
print('Number of tokens after normalization:',count_all_tokens(normalized_texts),end='\n\n')

lemmatized_texts = []
for text in normalized_texts:
  lemmatized_texts.append(lemmatize(text))
print('Lemmatized texts:\n',lemmatized_texts)
print('Number of tokens after lemmatization:',count_all_tokens(lemmatized_texts),end='\n\n')

preprocessed_texts = []
for text in lemmatized_texts:
  preprocessed_texts.append(remove_stopwords(text))
print('Texts after stopword removal:\n',preprocessed_texts)
print('Number of tokens after lemmatization:',count_all_tokens(preprocessed_texts),end='\n\n')


Tokenized texts:
 [['Situated', '22', 'km', 'away', 'from', 'Udaipur,', 'the', 'Ekling', 'Temple', 'is', 'one', 'of', 'the', 'most', 'famous', 'pilgrimage', 'sites', 'in', 'the', 'state', 'of', 'Rajasthan.', 'Ekling', 'is', 'believed', 'to', 'be', 'the', 'ruling', 'deity', 'of', 'the', 'princely', 'state', 'of', 'Mewar.', 'Dedicated', 'to', 'Lord', 'Shiva,', 'the', 'temple', 'was', 'built', 'in', '734', 'AD', 'and', 'has', 'been', 'repaired', 'and', 'modified', 'numerous', 'times.', 'A', 'four-faced', 'idol', 'of', 'Lord', 'Shiva', 'made', 'out', 'of', 'black', 'marble', 'rests', 'in', 'the', 'temple.', 'The', 'idol', 'is', '50', 'foot', 'tall', 'and', 'is', 'flanked', 'by', 'idols', 'of', 'Goddess', 'Parvati', 'and', 'her', 'two', 'sons,', 'Lord', 'Ganesha', 'and', 'Lord', 'Kartikeya.', 'The', 'main', 'temple', 'has', 'huge', 'silver', 'doors', 'depicting', 'the', 'two', 'sons', 'guarding', 'their', 'father.', 'One', 'must', 'visit', 'the', 'temple', 'during', 'the', 'festival', 'of',

## PART B: Inverted Index construction and using different data structures


In [40]:
# Constructing inverted index using hashmap of sets (Data structure type 1)
def inverted_index_construction(texts):
  postings = defaultdict(set)
  for idx, text in enumerate(texts):
    for token in text:
      postings[token].add(idx)
  return postings

In [41]:
# Generating postings list of hashmap of sets
postings = inverted_index_construction(preprocessed_texts)
postings

defaultdict(set,
            {'situated': {0, 20, 29, 51, 82, 87},
             '22': {0},
             'km': {0, 15, 21, 29, 51, 52, 82, 87, 91},
             'away': {0, 15, 16, 82, 89},
             'udaipur': {0, 58, 82, 89, 96},
             'ekling': {0},
             'temple': {0,
              5,
              7,
              16,
              42,
              62,
              72,
              74,
              75,
              79,
              82,
              86,
              87,
              88,
              89,
              93},
             'one': {0,
              1,
              2,
              4,
              6,
              7,
              8,
              9,
              10,
              15,
              17,
              20,
              23,
              26,
              29,
              30,
              36,
              37,
              38,
              40,
              41,
              42,
              46,
              48,
           

## PART C: Querying the generated postings lists.

In [42]:
# PART C: Query evaluation for generic expressions having AND,OR,NOT operators for hashmap of sets
# Precedence is given first to NOT > AND > OR operators.

def convert_to_exp(query,postings):
  exp = []
  q = query.split()
  for item in q:
    if item not in ['AND','OR','NOT']:
      exp.append(postings[item])
    else:
      exp.append(item)
  return exp

def parse_not(exp,preprocessed_texts,postings):
  res = []
  universal_set = set(range(len(preprocessed_texts)))
  flag = False
  for i in range(len(exp)):
    if flag==True:
      res.append(universal_set - exp[i])
      flag = False
    elif exp[i]=='NOT':
      flag=True
    else:
      res.append(exp[i])
  return res

def parse_and(exp,postings):
  res = []
  i=0
  while i<len(exp):
    if exp[i]=='AND':
     op1 = res.pop()
     op2 = exp[i+1]
     res.append(op1 & op2)
     i+=2
    else:
     res.append(exp[i])
     i+=1
  return res

def parse_or(exp,postings):
  res = []
  i=0
  while i<len(exp):
    if exp[i]=='OR':
     op1 = res.pop()
     op2 = exp[i+1]
     res.append(op1 | op2)
     i+=2
    else:
     res.append(exp[i])
     i+=1
  return res

#Main query function for hashmap of sets
def query_inverted_index(query,postings):
  exp = convert_to_exp(query,postings)
  exp = parse_not(exp,preprocessed_texts,postings)
  exp = parse_and(exp,postings)
  exp = parse_or(exp,postings)
  return exp[0]

In [32]:
#Examples for query type 1 for hashmap of sets
queries = ['life AND country AND war', 'saffron AND word AND mean']

for query in queries:
  s = query_inverted_index(query,postings)
  print('Query:',query)
  print('Query result: \nSet of document IDs:',s)
  docs = []
  for item in list(s):
    docs.append(files[item])
  print('Name of corresponding documents:',docs,end='\n\n')

Query: life AND country AND war
Query result: 
Set of document IDs: {24, 26}
Name of corresponding documents: ['national-war-memorial.txt', 'india-gate.txt']

Query: saffron AND word AND mean
Query result: 
Set of document IDs: {25}
Name of corresponding documents: ['sheer-korma.txt']



In [43]:
#Examples for query type 2 for hashmap of sets
queries = ['life OR country AND NOT war', 'saffron OR word AND NOT mean']

for query in queries:
  s = query_inverted_index(query,postings)
  print('Query:',query)
  print('Query result: \nSet of document IDs:',s)
  docs = []
  for item in list(s):
    docs.append(files[item])
  print('Name of corresponding documents:',docs,end='\n\n')

Query: life OR country AND NOT war
Query result: 
Set of document IDs: {64, 97, 68, 41, 48, 51, 24, 26, 92, 29, 62}
Name of corresponding documents: ['jute-decor.txt', 'kantha.txt', 'hyderabadi-biryani.txt', 'cubbon-park.txt', 'terracotta.txt', 'golconda-fort.txt', 'national-war-memorial.txt', 'india-gate.txt', 'mysore-silk.txt', 'mukutmanipur.txt', 'iskcon-temple.txt']

Query: saffron OR word AND NOT mean
Query result: 
Set of document IDs: {82, 68, 25, 75, 44}
Name of corresponding documents: ['nathdwara.txt', 'hyderabadi-biryani.txt', 'sheer-korma.txt', 'sankat-mochan-mandir.txt', 'mutton-biryani.txt']



In [45]:
# Constructing inverted index using hashmap of list (Data structure type 2)
def inverted_index_construction_list(texts):
  postings = defaultdict(list)
  for idx, text in enumerate(texts):
    for token in text:
      if idx not in postings[token]:
        postings[token].append(idx)
  return postings

In [46]:
# Generating postings list of hashmap of lists
postings = inverted_index_construction_list(preprocessed_texts)
postings

defaultdict(list,
            {'situated': [0, 20, 29, 51, 82, 87],
             '22': [0],
             'km': [0, 15, 21, 29, 51, 52, 82, 87, 91],
             'away': [0, 15, 16, 82, 89],
             'udaipur': [0, 58, 82, 89, 96],
             'ekling': [0],
             'temple': [0,
              5,
              7,
              16,
              42,
              62,
              72,
              74,
              75,
              79,
              82,
              86,
              87,
              88,
              89,
              93],
             'one': [0,
              1,
              2,
              4,
              6,
              7,
              8,
              9,
              10,
              15,
              17,
              20,
              23,
              26,
              29,
              30,
              36,
              37,
              38,
              40,
              41,
              42,
              46,
              48,
          

In [44]:
# PART C: Query evaluation for generic expressions having AND,OR,NOT operators for hashmap of lists
# Precedence is given first to NOT > AND > OR operators.

def list_difference(list1, list2):
    res = []
    for item in list1:
        if item not in list2:
            res.append(item)
    return res

def list_intersection(list1, list2):
    res = []
    for item in list1:
        if item in list2:
            res.append(item)
    return res

def list_union(list1, list2):
    res = list1[:]
    for item in list2:
        if item not in list1:
            res.append(item)
    return res

def convert_to_exp(query,postings):
  exp = []
  q = query.split()
  for item in q:
    if item not in ['AND','OR','NOT']:
      exp.append(postings[item])
    else:
      exp.append(item)
  return exp

def parse_not(exp,preprocessed_texts,postings):
  res = []
  universal_set = range(len(preprocessed_texts))
  flag = False
  for i in range(len(exp)):
    if flag==True:
      res.append(list_difference(universal_set, exp[i]))
      flag = False
    elif exp[i]=='NOT':
      flag=True
    else:
      res.append(exp[i])
  return res

def parse_and(exp,postings):
  res = []
  i=0
  while i<len(exp):
    if exp[i]=='AND':
     op1 = res.pop()
     op2 = exp[i+1]
     res.append(list_intersection(op1,op2))
     i+=2
    else:
     res.append(exp[i])
     i+=1
  return res

def parse_or(exp,postings):
  res = []
  i=0
  while i<len(exp):
    if exp[i]=='OR':
     op1 = res.pop()
     op2 = exp[i+1]
     res.append(list_union(op1,op2))
     i+=2
    else:
     res.append(exp[i])
     i+=1
  return res

#Main query function for hashmap of lists
def query_inverted_index_list(query,postings):
  exp = convert_to_exp(query,postings)
  exp = parse_not(exp,preprocessed_texts,postings)
  exp = parse_and(exp,postings)
  exp = parse_or(exp,postings)
  return exp[0]

In [39]:
#Examples for query type 1 for hashmap of lists
queries = ['life AND country AND war', 'saffron AND word AND mean']

for query in queries:
  s = query_inverted_index_list(query,postings)
  print('Query:',query)
  print('Query result: \nList of document IDs:',s)
  docs = []
  for item in list(s):
    docs.append(files[item])
  print('Name of corresponding documents:',docs,end='\n\n')

Query: life AND country AND war
Query result: 
List of document IDs: [24, 26]
Name of corresponding documents: ['national-war-memorial.txt', 'india-gate.txt']

Query: saffron AND word AND mean
Query result: 
List of document IDs: [25]
Name of corresponding documents: ['sheer-korma.txt']



In [49]:
#Examples for query type 2 for hashmap of lists
queries = ['life OR country AND NOT war', 'saffron OR word AND NOT mean']

for query in queries:
  s = query_inverted_index_list(query,postings)
  print('Query:',query)
  print('Query result: \nSet of document IDs:',s)
  docs = []
  for item in list(s):
    docs.append(files[item])
  print('Name of corresponding documents:',docs,end='\n\n')

Query: life OR country AND NOT war
Query result: 
Set of document IDs: [24, 26, 51, 62, 97, 29, 41, 48, 64, 68, 92]
Name of corresponding documents: ['national-war-memorial.txt', 'india-gate.txt', 'golconda-fort.txt', 'iskcon-temple.txt', 'kantha.txt', 'mukutmanipur.txt', 'cubbon-park.txt', 'terracotta.txt', 'jute-decor.txt', 'hyderabadi-biryani.txt', 'mysore-silk.txt']

Query: saffron OR word AND NOT mean
Query result: 
Set of document IDs: [25, 44, 68, 75, 82]
Name of corresponding documents: ['sheer-korma.txt', 'mutton-biryani.txt', 'hyderabadi-biryani.txt', 'sankat-mochan-mandir.txt', 'nathdwara.txt']

